# Cleanup on Row 7

1. Clean the File
2. Handle Corrupt Rows
3. Explore Data Set Bias

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

### Helper Display Functions

In [5]:
def display_df(df, nrows=10, ncols=None):
    with pd.option_context('display.max_rows', nrows, 'display.max_columns', ncols):
        display (df)
def print_row(df, row):
    for ctr,i in enumerate(df.iloc[row]):
        print (str(df.columns[ctr])+": "+str(i))

## Importing the Data

In [52]:
df_raw = pd.read_csv("../dirty_sample_small.csv")

ParserError: Error tokenizing data. C error: Expected 48 fields in line 652242, saw 49


Some rows have an extra field. We shall investigate what this is. First we will import the file in a more managable way, capable of handling these erroneous lines.

In [55]:
df_raw = pd.DataFrame([line.strip().split(',') for line in open('../dirty_sample_small.csv', 
                                                                'r', encoding="ISO-8859-1")])

In [63]:
len(df_raw)

661487

In [18]:
#gets ratio of NaNs for each column
def stats_NaN(df):
    df_stats = pd.DataFrame(index=[df.columns], columns=["NaN Ratio"])
    for col in df.columns:
        df_stats["NaN Ratio"][col] = df[col].isna().sum()/len(df) #NaN ratio
    return df_stats.sort_values(by=['NaN Ratio'])

#helps decide what to make dummies for and what to drop
def stats_unique(df, labels):
    df_counts = pd.DataFrame(index=labels,columns=['Unique Count'])
    for l in labels:
        df_counts['Unique Count'][l] = df[l].nunique()
    return df_counts

In [50]:
stats_NaN(df_raw)

,NaN Ratio
0,0
1,0
2,0
3,0.000270602
4,0.000498876
5,0.000757384
6,0.00102496
7,0.00126231
8,0.00155407
9,0.00181712


We see that that the extra has 98.6% NaNs. This is enough to drop it. However we want to do our due diligence and investigate what the values that are not NaN are. Our suspicion is that it contains the empty string "". So we will query this dataset on the extra field for values that are not NaN or not the empty string.

In [56]:
df_raw[(pd.notna(df_raw[48]) & (df_raw[48] != ''))][48]

Series([], Name: 48, dtype: object)

We see that this returns nothing. Therefore everything in the extra field is either NaN or the empty string, so there is absolutely no value in keeping it and we will drop it. But first, lets see how many of these corrupt erroneous lines there are. We know that the construction of df_raw line by line as we did above is indifferent to the amount of columns in each row. Most of the time it saw 48, in the later rows it saw 49 because 

In [65]:
len(df_raw[((df_raw[48] == ''))][48])

9160

In [ ]:
df_c